In [2]:
import numpy as np
from scipy.special import erfc

class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
import numpy as np
from scipy.special import erfc

def S2B(symbol, table):
    b = table[symbol]
    B = np.array(list(b)).astype(np.float32)
    return B

def probB(x, N0):
    d1 = abs(-1-x)
    d2 = abs(1-x)
    p0 = np.e**(-d1/N0)/(np.e**(-d1/N0)+np.e**(-d2/N0))
    p1 = 1-p0
    return [p0, p1]

def probS(S, N0, q, table, M):
    B = np.zeros((q, 2))
    for i in range(q):
        B[i] = probB(S[i], N0)
    P = np.zeros(M)
    for i in range(M):
        Bi = S2B(i, table)
        p = 1
        for j in range(q):
            p *=B[j, int(Bi[j])]
        P[i] = p
    m = np.max(P)
    P = np.log10(m/P)
    return P

def BPSK(message, M, table, SNRdB):
    Eb = 1
    N0 = Eb/10**(SNRdB/10)
    q = int(np.log2(M))
    Ns = len(message)
    Nb = Ns*q
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    noise = noise.reshape((Ns, q))
    X = np.zeros_like(noise)
    for i in range(Ns):
        X[i]= S2B(message[i], table)
    X[X==0]=-1
    Y = X+noise
    output = np.zeros((Ns, M))
    for i in range(Ns):
        output[i] = probS(Y[i], N0, q, table, M)
    return output



def HardDicision(Message):
    return np.argmin(Message, axis=1)



In [ ]:
def BPSK_TEST(x0, SNRdB):
    x = x0.copy()
    x[x==0]=-1
    Eb = 1
    N0 = Eb/10**(SNRdB/10)
    Nb = x.shape[0]
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    y = x+noise
    y[y<=0]=0
    y[y>0]=1
    z = np.zeros_like(x0)
    z[x0!=y]=1
    return np.sum(z)


    
    
def BER(SNRdbs):
    max_bits = 1e8
    for SNRdb in SNRdbs:
        x = np.random.randint(0, 2, int(max_bits))
        errors = BPSK_TEST(x, SNRdb)
        ber = np.log10(errors/max_bits)
        print(SNRdb,  ber)

BER([1, 2, 4, 6 ,8 ,10, 12]

In [4]:
gl = GF([5,  2, 0])
from numpy import roots
def can5(n):
    return np.real(roots([1]+[0]*(4)+[-n])[-1])

for SNRdB in range(16):
    N = 0
    E = 0
    F = 100
    while E<100 and N<1e8:
        X = np.random.randint(0, 32, F)
        Y = HardDicision(BPSK(X, 32, gl.table, SNRdB))
        Z = np.zeros_like(X)
        Z[X!=Y]=1
        E+=np.sum(Z)
        N+=F
    Pf = E/N
    Pb = 1-can5(1-Pf)
    print(SNRdB, np.log10(Pb))

0 -1.1455369887761429
1 -1.2238844389454564
2 -1.4142857413388823
3 -1.6329924718025257
4 -1.8930008221525148
5 -2.229315204612331
6 -2.5788486719982835
7 -3.12155934775565
8 -3.7409939029516286
9 -4.5360910713761555
10 -5.400983279818313


KeyboardInterrupt: 